In [1]:
import yaml
import pyodbc
from market_growth_analysis.etl.stagging import *
import pandas as pd
import numpy as np
import pyodbc

# Load the YAML file
with open('../../conf/global.yml', 'r') as f:
    columns = yaml.safe_load(f)

# Load the YAML file
with open('../../conf/local.yml', 'r') as f:
    credentials = yaml.safe_load(f)

In [2]:
ratios_df = pd.read_csv("../../data/load_04/ratios_df.csv", index_col=0)
income_statememnt_df = pd.read_csv("../../data/load_04/income_statememnt_df.csv", index_col=0)
balance_sheet_df = pd.read_csv("../../data/load_04/balance_sheet_df.csv", index_col=0)
cash_flow_statement_df = pd.read_csv("../../data/load_04/cash_flow_statement_df.csv", index_col=0)
prices_df = pd.read_csv("../../data/load_04/prices_df.csv", index_col=0)
dim_grouped_df = pd.read_csv("../../data/load_04/dim_grouped_df.csv", index_col=0)

In [3]:
financial_dict = {
'FACT_RATIOS': ratios_df,
'FACT_INCOME_STATEMENT': income_statememnt_df,
'FACT_BALANCE_SHEET': balance_sheet_df,
'FACT_CASH_FLOW_STATEMENT': cash_flow_statement_df,
'FACT_PRICES': prices_df,
'DIM_COMPANY': dim_grouped_df
}

In [4]:
driver = credentials['warehouse_db']['driver']
server = credentials['warehouse_db']['server']
database = credentials['warehouse_db']['database']
trusted = credentials['warehouse_db']['trusted_connection']



In [5]:
conn = pyodbc.connect('Driver={SQL Server};'
                      f'Server={server};'
                      f'Database={database};'
                      'Trusted_Connection=yes;')

In [6]:
cursor = conn.cursor()

In [7]:
def create_table_from_dataframe(df, table_name, cursor):
    # Drop the table if it exists
    drop_table_query = f'DROP TABLE IF EXISTS {table_name};'
    cursor.execute(drop_table_query)

    # Generate the column definitions with matching data types
    columns = []
    for column_name, column_type in df.dtypes.items():
        if column_name == 'PK':
            columns.append(f'[{column_name}] [int] PRIMARY KEY')
        elif column_type == 'object':
            columns.append(f'[{column_name}] VARCHAR(MAX) NULL')
        elif column_type == 'int64':
            columns.append(f'[{column_name}] INT NULL')
        elif column_type == 'float64':
            columns.append(f'[{column_name}] FLOAT NULL')
        elif column_type == 'datetime64[ns]':
            columns.append(f'[{column_name}] DATETIME NULL')
        elif column_type == 'bool':
            columns.append(f'[{column_name}] BIT NULL')

    # Create the table query
    create_table_query = f'CREATE TABLE {table_name} (\n'
    create_table_query += ',\n'.join(columns)
    create_table_query += '\n) ON [PRIMARY]'

    print(create_table_query)
    # Execute the CREATE TABLE statement
    cursor.execute(create_table_query)



def insert_data_into_table(df, table_name, cursor):
    # Generate the INSERT INTO statement
    columns = ', '.join(df.columns)
    placeholders = ', '.join(['?' for _ in df.columns])
    insert_query = f'INSERT INTO [{table_name}] ({columns}) VALUES ({placeholders});'

    # Insert the data row by row
    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))


In [8]:
for table in financial_dict.keys():
    create_table_from_dataframe(financial_dict[table], table, cursor)


CREATE TABLE FACT_RATIOS (
[PK] [int] PRIMARY KEY,
[Enterprise Value (EV)] FLOAT NULL,
[EV / Revenue] FLOAT NULL,
[EV / EBITDA] FLOAT NULL,
[EV / EBIT] FLOAT NULL,
[EV / Invested Capital] FLOAT NULL,
[Free Cash Flow (FCF)] FLOAT NULL,
[EV / Free Cash Flow] FLOAT NULL,
[EV / FCF] FLOAT NULL,
[P/E] FLOAT NULL,
[P/S] FLOAT NULL,
[P/CF] FLOAT NULL
) ON [PRIMARY]
CREATE TABLE FACT_INCOME_STATEMENT (
[PK] [int] PRIMARY KEY,
[Cash On Hand] FLOAT NULL,
[Receivables] FLOAT NULL,
[Inventory] FLOAT NULL,
[Pre-Paid Expenses] FLOAT NULL,
[Other Current Assets] FLOAT NULL,
[Total Current Assets] FLOAT NULL,
[Property, Plant, And Equipment] FLOAT NULL,
[Long-Term Investments] FLOAT NULL,
[Goodwill And Intangible Assets] FLOAT NULL,
[Other Long-Term Assets] FLOAT NULL,
[Total Long-Term Assets] FLOAT NULL,
[Total Assets] FLOAT NULL,
[Total Current Liabilities] FLOAT NULL,
[Long Term Debt] FLOAT NULL,
[Other Non-Current Liabilities] FLOAT NULL,
[Total Long Term Liabilities] FLOAT NULL,
[Total Liabilitie

In [9]:
for table in financial_dict.keys():
    insert_data_into_table(financial_dict[table], table, cursor)

conn.close

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Sintaxis incorrecta cerca de 'Value'. (102) (SQLExecDirectW); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]No se puede preparar la instrucción o instrucciones. (8180)")